In [25]:
from datetime import datetime as dt
from datetime import timedelta
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

# 日付条件の設定
strdt = dt.strptime("20180101", '%Y%m%d')  # 開始日
enddt = dt.strptime("20191231", '%Y%m%d')  # 終了日

days_num = (enddt - strdt).days + 1

In [2]:
datelist = []
for i in range(days_num):
    datelist.append(strdt + timedelta(days=i))

In [3]:
address=[]
for d in datelist:
    url="https://www.reuters.com/resources/archive/jp/"+ d.strftime("%Y%m%d")+".html"
    address.append(url)

In [4]:
p = re.compile(r"<[^>]*?>")

In [5]:
import pickle
def makefile(what,filename):
    with open(filename,"wb") as f3:
        pickle.dump(what,f3)
def readfile(filename):
    with open(filename,"rb") as f4:
        ans=pickle.load(f4)
    return ans

In [6]:
url = address[0]
html = requests.get(url)
html.encoding = 'shift-jis'
soup = BeautifulSoup(html.text, 'html.parser')
url2=soup.find_all("body")
url3=url2[0].find_all("div",{"class":"headlineMed"})
result = []
for word in url3:
    string=p.sub("",str(word))
    result.append(string)

In [11]:
URL = pd.DataFrame(index=[],columns=[])
texts=dict()
n=0
for url in address:
    if url[-13:-5] in texts:
        pass
    else:
        try:
            for i in range(0,len(url)):
                html = requests.get(url)
                html.encoding = 'utf-8'
                soup = BeautifulSoup(html.text, 'html.parser')
                url2=soup.find_all("body")
                url3=url2[0].find_all("div",{"class":"headlineMed"})
                #link = url3[i].find("a")
                #tmp_se = pd.Series( [url[-13:-5], link.get('href')])
                #URL = URL.append( tmp_se, ignore_index=True )
                result = []
            for word in url3:
                string=p.sub("",str(word))
                result.append(string)
            texts[url[-13:-5]]=result
        except:
            texts[url[-13:-5]]=[]
        n+=1
        if n%100==0:
            print(n)

100
200
300
400
500
600
700


In [12]:
URL = pd.read_csv('URL2.csv')
URL = URL.drop("Unnamed: 0", axis=1)

In [23]:
URL

,0,1
0,20180101,http://JP.reuters.com/article/idJP201801010100...
1,20180101,http://JP.reuters.com/article/idJP201801010100...
2,20180101,http://JP.reuters.com/article/idJP201801010100...
3,20180101,http://JP.reuters.com/article/usa-potus-idJPKB...
4,20180101,http://JP.reuters.com/article/global-poy-stori...
...,...,...
86267,20191209,http://JP.reuters.com/article/idJP201912100100...
86268,20191209,http://JP.reuters.com/article/idJPL4N28K1UA
86269,20191209,http://JP.reuters.com/article/idJP201912100100...
86270,20191209,http://JP.reuters.com/article/%E5%BE%93%E6%A5%...


## title

In [14]:
TITLE = pd.DataFrame(list(texts.items()),columns=['date', 'TITLEs'])
title = pd.DataFrame(index=[],columns=['date', 'TITLE'])

for i in range(0,len(TITLE['TITLEs'])):
    for l in range(0,len(TITLE['TITLEs'][i])):
        tmp_se = pd.Series( [TITLE['date'][i], TITLE['TITLEs'][i][l]], index=title.columns )
        title = title.append( tmp_se, ignore_index=True )

In [15]:
title

,date,TITLE
0,20180101,イランデモ、死者計１３人に 11:38午後 UTC
1,20180101,Ａ東京、琉球が勝つ 11:14午後 UTC
2,20180101,香港民主派が元日デモ 9:22午後 UTC
3,20180101,写真が語る2017年：物議を呼んだ米大統領就任式の全景写真 6:56午後 UTC
4,20180101,写真が語る2017年：ロヒンギャ難民女性、決死の脱出 6:55午後 UTC
...,...,...
176621,20191209,Breakingviews TV: PG&amp;E pays up 2:52午前 UTC
176622,20191209,"M&amp;A Monday in healthcare: Merck, Sanofi, U..."
176623,20191209,South African activist crowned Miss Universe 2...
176624,20191209,Just Eat takeover battle hots up 1:10午前 UTC


In [16]:
title.to_csv('TITLE-split2018-2019.csv')

## NEWS

In [26]:
news = pd.DataFrame(index=[],columns=['date', 'NEWS'])

for i in range(0,len(URL)):
    html = requests.get(URL['1'][i])
    html.encoding = 'utf-8'
    soup = BeautifulSoup(html.text, 'html.parser')
    url2=soup.find_all("body")
    url3=url2[0].find_all("div",{"class":"headlineMed"})
    url4 = soup.find(class_="Paragraph-paragraph-2Bgue ArticleBody-para-TD_9x")
    try:
        tmp_se = pd.Series( [URL['0'][i], url4.get_text()], index=news.columns )
        news = news.append( tmp_se, ignore_index=True )
    except AttributeError as e:
        tmp_se = pd.Series( [URL['0'][i], np.nan], index=news.columns )
        news = news.append( tmp_se, ignore_index=True )

In [27]:
news.to_csv('NEWS-split2018-2019.csv')

In [28]:
URL['NEWS'] = news['NEWS']
URL = URL.rename(columns={0: 'date',1: 'URL'})

In [29]:
URL.to_csv('URL2.csv')
#URL.to_csv('URL2.csv', mode='a', header=False)